In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, random, shutil, subprocess
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# =====================================
# 이미지 zip 파일은 celeba 폴더 에 넣기
# =====================================

img_zip_file = 'inference_imgs_dc50.zip'

In [ ]:
# ==============
# Import ESRGAN
# ==============

!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights

!pip uninstall -y torch torchvision
!pip install torch==2.0.1 torchvision==0.15.2

In [ ]:
# ====================
# 이미지 압축파일 해제
# ====================

source_path = f'/content/drive/MyDrive/celeba/{img_zip_file}'
copy_path = '/content/example.zip'
extract_path = '/content/extracted'

folder_name = os.path.splitext(img_zip_file)[0]

shutil.copy(source_path, copy_path)

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

!unzip -q {copy_path} -d {extract_path}

inner_dir = os.path.join(extract_path, folder_name)

if os.path.isdir(inner_dir):
    for filename in os.listdir(inner_dir):
        src_path = os.path.join(inner_dir, filename)
        dst_path = os.path.join(extract_path, filename)
        shutil.move(src_path, dst_path)
    os.rmdir(inner_dir)

In [ ]:
# =============================================
# 랜덤 이미지 1개 선택해서 Upscaling Inference
# =============================================

selected = random.choice(os.listdir(extract_path))
name, ext = os.path.splitext(selected)
upscaled = f"{name}_out{ext}"
input_path = os.path.join(extract_path, selected)
subprocess.run(['python', 'inference_realesrgan.py', '-n', 'RealESRGAN_x4plus', '-i', input_path, '--face_enhance'])
original = Image.open(input_path).resize((299, 299))
upscaled = Image.open(os.path.join('results', upscaled)).resize((299, 299))

plt.subplot(1, 2, 1)
plt.imshow(original)
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(upscaled)
plt.axis('off')
plt.show()

In [ ]:
# ===================================
# 이미지 N 개 선택해서 Upscaling 하고
# 해당 이미지들을 압축파일로 만듬
# resize 는 안함
# 압축파일은 /content/ 에 생김
# ===================================

import zipfile

N = 5000

images = [f for f in os.listdir(extract_path) if f.lower().endswith(('.png','jpg','jpeg','bmp','gif'))]
selected = random.sample(images, N)
input_dir = '/content/selected_inputs'
os.makedirs(input_dir, exist_ok=True)
for f in selected:
    shutil.copy(os.path.join(extract_path, f), input_dir)
subprocess.run(['python', 'inference_realesrgan.py', '-n', 'RealESRGAN_x4plus', '-i', input_dir, '--face_enhance'])
result_path = '/content/Real-ESRGAN/results'
with zipfile.ZipFile('/content/output.zip', 'w') as z:
    for f in os.listdir(result_path):
        z.write(os.path.join(result_path, f), f)
shutil.rmtree(input_dir)